# PhoBERT NER - Xử lý dữ liệu

Notebook này chuyển đổi dữ liệu từ format CSV sang format JSON phù hợp với PhoBERT.

## Input
- File: `../data/raw/train_data_ner.csv`
- Format: CSV với columns [sentence_id, token, label]

## Output
- `../data/processed/train_phobert.json` - Training set (80%)
- `../data/processed/val_phobert.json` - Validation set (20%)

In [1]:
import pandas as pd
import json
from collections import defaultdict
import random

# Set random seed để reproducible
random.seed(42)

## 1. Đọc dữ liệu CSV

In [21]:
# Đọc file CSV
df = pd.read_csv("../../data/raw/raw_data_ner.csv")

# Loại bỏ khoảng trắng thừa từ tên cột
df.columns = df.columns.str.strip()

# Loại bỏ các dòng NaN
df = df.dropna(subset=['sentence_id', 'token', 'label'])

print(f"✓ Đã đọc {len(df)} tokens từ file CSV")
print(f"✓ Số lượng câu: {df['sentence_id'].nunique()}")
print("\nDữ liệu mẫu:")
print(df.head(10))

✓ Đã đọc 10642 tokens từ file CSV
✓ Số lượng câu: 1001

Dữ liệu mẫu:
   sentence_id token      label
0            1  Chào          O
1            1   bác          O
2            1    sĩ          O
4            1   gần          O
5            1   đây          O
6            1   tôi          O
7            1    bị          O
8            1  chảy  B-SYMPTOM
9            1  nước  I-SYMPTOM
10           1   mũi  I-SYMPTOM


## 2. Chuyển đổi sang format PhoBERT

Format PhoBERT cần:
```json
{
  "tokens": ["Tôi", "bị", "đau", "đầu"],
  "ner_tags": ["O", "O", "B-SYMPTOM", "I-SYMPTOM"]
}
```

In [22]:
def convert_to_phobert_format(dataframe):
    """
    Chuyển đổi DataFrame sang format JSON cho PhoBERT
    
    Args:
        dataframe: DataFrame với columns [sentence_id, token, label]
    
    Returns:
        List of dict với format {"tokens": [...], "ner_tags": [...]}
    """
    phobert_data = []
    
    # Nhóm theo sentence_id
    grouped = dataframe.groupby('sentence_id')
    
    for sentence_id, group in grouped:
        tokens = group['token'].tolist()
        labels = group['label'].tolist()
        
        # Đảm bảo token là string
        tokens = [str(token).strip() for token in tokens]
        labels = [str(label).strip() for label in labels]
        
        phobert_data.append({
            "tokens": tokens,
            "ner_tags": labels
        })
    
    return phobert_data

# Chuyển đổi
all_data = convert_to_phobert_format(df)

print(f"✓ Đã chuyển đổi {len(all_data)} câu sang format PhoBERT")
print("\nVí dụ dữ liệu sau khi chuyển đổi:")
for i, item in enumerate(all_data[:3]):
    print(f"\nCâu {i+1}:")
    print(f"  Tokens: {item['tokens']}")
    print(f"  Labels: {item['ner_tags']}")

✓ Đã chuyển đổi 1001 câu sang format PhoBERT

Ví dụ dữ liệu sau khi chuyển đổi:

Câu 1:
  Tokens: ['Chào', 'bác', 'sĩ', 'gần', 'đây', 'tôi', 'bị', 'chảy', 'nước', 'mũi', 'liên', 'tục', 'và', 'ho', 'khàn', 'kéo', 'dài']
  Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM']

Câu 2:
  Tokens: ['Tôi', 'bị', 'đau', 'mình', 'mẩy', 'và', 'nhức', 'mỏi', 'toàn', 'thân']
  Labels: ['O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM']

Câu 3:
  Tokens: ['Tôi', 'bị', 'ho', 'kéo', 'dài', 'và', 'bị', 'nghẹt', 'mũi']
  Labels: ['O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM']


## 3. Thống kê labels

In [23]:
# Đếm số lượng mỗi loại label
label_counts = defaultdict(int)
unique_labels = set()

for item in all_data:
    for label in item['ner_tags']:
        label_counts[label] += 1
        unique_labels.add(label)

print("📊 Thống kê labels:")
print(f"Tổng số labels duy nhất: {len(unique_labels)}")
print("\nPhân bố labels:")
for label, count in sorted(label_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {label}: {count} tokens ({count/sum(label_counts.values())*100:.1f}%)")

print(f"\n✓ Danh sách labels: {sorted(unique_labels)}")

📊 Thống kê labels:
Tổng số labels duy nhất: 8

Phân bố labels:
  O: 4460 tokens (41.9%)
  I-SYMPTOM: 4180 tokens (39.3%)
  B-SYMPTOM: 1920 tokens (18.0%)
  I-DISEASE: 38 tokens (0.4%)
  Tôi: 21 tokens (0.2%)
  B-DISEASE: 20 tokens (0.2%)
  I-I-SYMPTOM: 2 tokens (0.0%)
  I-SYMPTEM: 1 tokens (0.0%)

✓ Danh sách labels: ['B-DISEASE', 'B-SYMPTOM', 'I-DISEASE', 'I-I-SYMPTOM', 'I-SYMPTEM', 'I-SYMPTOM', 'O', 'Tôi']


## 4. Split Train/Validation (80/20)

In [24]:
# Shuffle data
random.shuffle(all_data)

# Tính split point
split_idx = int(len(all_data) * 0.8)

train_data = all_data[:split_idx]
val_data = all_data[split_idx:]

print(f"✓ Training set: {len(train_data)} câu ({len(train_data)/len(all_data)*100:.1f}%)")
print(f"✓ Validation set: {len(val_data)} câu ({len(val_data)/len(all_data)*100:.1f}%)")

✓ Training set: 800 câu (79.9%)
✓ Validation set: 201 câu (20.1%)


## 5. Lưu dữ liệu vào file JSON

In [25]:
import os

# Tạo thư mục nếu chưa có
os.makedirs("../../data/processed", exist_ok=True)

# Lưu training set
train_path = "../../data/processed/train_phobert.json"
with open(train_path, 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

print(f"✓ Đã lưu training set vào: {train_path}")

# Lưu validation set
val_path = "../../data/processed/val_phobert.json"
with open(val_path, 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

print(f"✓ Đã lưu validation set vào: {val_path}")

✓ Đã lưu training set vào: ../../data/processed/train_phobert.json
✓ Đã lưu validation set vào: ../../data/processed/val_phobert.json


## 6. Kiểm tra dữ liệu đã lưu

In [26]:
# Đọc lại để verify
with open(train_path, 'r', encoding='utf-8') as f:
    loaded_train = json.load(f)

with open(val_path, 'r', encoding='utf-8') as f:
    loaded_val = json.load(f)

print("✓ Kiểm tra dữ liệu đã lưu:")
print(f"  Training: {len(loaded_train)} câu")
print(f"  Validation: {len(loaded_val)} câu")

print("\n✅ Hoàn thành xử lý dữ liệu!")
print("\n📝 Bước tiếp theo: Chạy notebook 02_train_phobert_ner.ipynb để train model")

✓ Kiểm tra dữ liệu đã lưu:
  Training: 800 câu
  Validation: 201 câu

✅ Hoàn thành xử lý dữ liệu!

📝 Bước tiếp theo: Chạy notebook 02_train_phobert_ner.ipynb để train model
